In [32]:
import re
import nltk
import pandas as pd
from datetime import datetime
from sklearn.model_selection import train_test_split


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrew\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Load Dataset and Create Copy

In [33]:
# Load dataset
df = pd.read_csv("../data/raw/letterboxd-reviews.csv", encoding='latin1')

# Work on a copy
clean_df = df.copy()

In [34]:
clean_df.head(10)

,Movie name,Release Year,Rating,Reviewer name,Review date,Review,Comment count,Like count
0,Aftersun (2022),2022,â??â??â??â??Â½,Tuomas,12-Jan-20,This review may contain spoilers.,130,"22,44 6 likes"
1,Joker (2019),2019,â??â??â??â??â??,Joao,20-Dec-22,if youâ??ve never swam in the ocean then of co...,1.8K,"22,032 likes"
2,Puss in Boots: The Last Wish (2022),2022,â??Â½,NicoPico,15-Sep-22,Puss in Boots: Into the Pussy-Verse,6 2,"21, 6 6 6 likes"
3,The Banshees of Inisherin (2022),2022,â??â??â??â??â??,Ella Kemp,8-Apr-22,I will NOT leave my donkey outside when Iâ??m sad,NaN,"21, 6 09 likes"
4,Everything Everywhere All at Once (2022),2022,â??â??Â½,CosmonautMarkie,14-Aug-19,Watch it and have fun before film Twitter tell...,355,"20, 6 88 likes"
5,Parasite (2019),2019,â??â??â??â??,Philbert Dy,6 /21/2022,This review may contain spoilers.,169,"20,346 likes"
6,Nope (2022),2022,â??â??â??,tyler,6 / 6 /2022,this movie does for cloud spotting what did fo...,NaN,"20,3 6 8 likes"
7,Thor: Love and Thunder (2022),2022,NaN,24framesofnick,13-Oct-22,Great first draft! Excited to see when itâ??s ...,98,"20,316 likes"
8,The Menu (2022),2022,â??â??â??â??â??,Jay,25-Oct-21,This review may contain spoilers.,6 5,"20,359 likes"
9,Dune (2021),2021,â??â??â??â??,kÃ¡rsten,9-Aug-19,got the 4D experience by forgetting to drink w...,4,"19, 6 24 likes"


### Clean Column Names
Replace spaces and remove capitalization to make accessing columns easier.

In [35]:
# Clean column names
clean_df.columns = (
    clean_df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

### Fix Movie Rating
Add numeric_rating by counting the broken star rating symbols

In [36]:
def fix_rating(r):
    if pd.isna(r):
        return None
    s = str(r)

    stars = s.count("â??")
    half = 0.5 if "Â½" in s else 0
    return stars + half

clean_df["numeric_rating"] = clean_df["rating"].apply(fix_rating)

# Sanity check
clean_df[["rating", "numeric_rating"]].head(10)

,rating,numeric_rating
0,â??â??â??â??Â½,4.5
1,â??â??â??â??â??,5.0
2,â??Â½,1.5
3,â??â??â??â??â??,5.0
4,â??â??Â½,2.5
5,â??â??â??â??,4.0
6,â??â??â??,3.0
7,NaN,NaN
8,â??â??â??â??â??,5.0
9,â??â??â??â??,4.0


### Clean "like_count" and "comment_count"

In [37]:
def clean_count(s):
    if pd.isna(s):
        return None
    
    s = str(s).lower()
    s = s.replace("likes", "")
    s = s.replace(",", " ")
    s = re.sub(r"[^0-9.k]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()

    if not s:
        return None
    
    match = re.search(r"(\d+(\.\d+)?)\s*k", s)
    if match:
        num = float(match.group(1)) * 1000
        return int(num)
    
    digits = re.findall(r"\d+", s)
    if digits:
        num_str = "".join(digits)
        try:
            return int(num_str)
        except ValueError:
            return None
        
    return None

clean_df["like_count"] = clean_df["like_count"].apply(clean_count)
clean_df["comment_count"] = clean_df["comment_count"].apply(clean_count)

### Clean and Convert "review_date" to datetime
Review dates seem to be in completely random messy formats, so they need to be cleaned and converted to datetime.

In [38]:
def date_cleaner(date_str):
    if pd.isna(date_str):
        return None

    s = str(date_str).strip()

    # Remove the weird spaces
    s = re.sub(r"\s+", "", s)

    # Try the formats observed in the dataset
    for fmt in ("%d-%b-%y", "%d-%b-%Y", "%m/%d/%Y", "%m/%d/%y"):
        try:
            return datetime.strptime(s, fmt)
        except ValueError:
            pass

    # Handle the messy slash formats like "6 /21/2022" or "6/ 6/2022"
    try:
        return pd.to_datetime(s, errors="coerce")
    except:
        return None

clean_df["review_date_clean"] = clean_df["review_date"].apply(date_cleaner)

# Sanity check
clean_df[["review_date", "review_date_clean"]].head(10)

,review_date,review_date_clean
0,12-Jan-20,2020-01-12
1,20-Dec-22,2022-12-20
2,15-Sep-22,2022-09-15
3,8-Apr-22,2022-04-08
4,14-Aug-19,2019-08-14
5,6 /21/2022,2022-06-21
6,6 / 6 /2022,2022-06-06
7,13-Oct-22,2022-10-13
8,25-Oct-21,2021-10-25
9,9-Aug-19,2019-08-09


### Clean Review

In [39]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_review(s):
    if pd.isna(s):
        return ""
    s = re.sub(r"<.*?>", " ", s)
    s = re.sub(r"[^a-zA-Z\s]", " ", s)
    s = s.lower()
    tokens = [lemmatizer.lemmatize(w) for w in s.split() if w not in stop_words]
    return " ".join(tokens)

clean_df["review_clean"] = clean_df["review"].apply(clean_review)

# Sanity check
clean_df[["review", "review_clean"]].head(10)

,review,review_clean
0,This review may contain spoilers.,review may contain spoiler
1,if youâ??ve never swam in the ocean then of co...,never swam ocean course pool seems deep
2,Puss in Boots: Into the Pussy-Verse,pus boot pussy verse
3,I will NOT leave my donkey outside when Iâ??m sad,leave donkey outside sad
4,Watch it and have fun before film Twitter tell...,watch fun film twitter tell overrated
5,This review may contain spoilers.,review may contain spoiler
6,this movie does for cloud spotting what did fo...,movie cloud spotting swimming
7,Great first draft! Excited to see when itâ??s ...,great first draft excited see finished
8,This review may contain spoilers.,review may contain spoiler
9,got the 4D experience by forgetting to drink w...,got experience forgetting drink water today wa...


### Clean "release_year" Column

In [40]:
def clean_release_year(s):
    if pd.isna(s):
        return pd.NA
    
    s = str(s).strip()

    s = re.sub(r"[^\d]", "", s)

    try:
        year = int(s)
    except ValueError:
        return pd.NA

    current_year = pd.Timestamp.now().year
    if 1880 <= year <= current_year + 1:
        return year
    else:
        return pd.NA

clean_df["release_year"] = clean_df["release_year"].apply(clean_release_year).astype("Int64")

### Handle Missing Values
Drop rows with missing "movie_name" and "review".

In [41]:
clean_df.isna().sum()
clean_df = clean_df.dropna(subset=["movie_name", "review"])

### Drop and Replace Original Columns with Clean Ones

In [42]:
# Drop and replace original columns with the clean ones
clean_df.drop(columns=["rating", "review_date", "review"], inplace=True)
clean_df.rename(columns={"numeric_rating": "rating", "review_date_clean": "review_date", "review_clean": "review"}, inplace=True)

### Date Type Conversions

In [43]:
# Convert "movie_name" to string
clean_df["movie_name"] = clean_df["movie_name"].astype("string")

# Convert "release_year" to int
# clean_df["release_year"] = pd.to_numeric(clean_df["release_year"], errors="coerce").astype("Int64")

# Convert "reviewer_name" to string
clean_df["reviewer_name"] = clean_df["reviewer_name"].astype("string")

# Convert "review" to string
clean_df["review"] = clean_df["review"].astype("string")


### Split the Dataset (Train / Validation / Test)

In [44]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)



### Save Cleaned Data

In [45]:
clean_df.to_csv("../data/processed/letterboxd_reviews_clean.csv", index=False)

train_df.to_csv("../data/processed/train.csv", index=False)
val_df.to_csv("../data/processed/val.csv", index=False)
test_df.to_csv("../data/processed/test.csv", index=False)

In [46]:
clean_df.head(10)

,movie_name,release_year,reviewer_name,comment_count,like_count,rating,review_date,review
0,Aftersun (2022),2022,Tuomas,130.0,22446.0,4.5,2020-01-12,review may contain spoiler
1,Joker (2019),2019,Joao,1800.0,22032.0,5.0,2022-12-20,never swam ocean course pool seems deep
2,Puss in Boots: The Last Wish (2022),2022,NicoPico,62.0,21666.0,1.5,2022-09-15,pus boot pussy verse
3,The Banshees of Inisherin (2022),2022,Ella Kemp,NaN,21609.0,5.0,2022-04-08,leave donkey outside sad
4,Everything Everywhere All at Once (2022),2022,CosmonautMarkie,355.0,20688.0,2.5,2019-08-14,watch fun film twitter tell overrated
5,Parasite (2019),2019,Philbert Dy,169.0,20346.0,4.0,2022-06-21,review may contain spoiler
6,Nope (2022),2022,tyler,NaN,20368.0,3.0,2022-06-06,movie cloud spotting swimming
7,Thor: Love and Thunder (2022),2022,24framesofnick,98.0,20316.0,NaN,2022-10-13,great first draft excited see finished
8,The Menu (2022),2022,Jay,65.0,20359.0,5.0,2021-10-25,review may contain spoiler
9,Dune (2021),2021,kÃ¡rsten,4.0,19624.0,4.0,2019-08-09,got experience forgetting drink water today wa...


In [47]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3560 entries, 0 to 4577
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   movie_name     3560 non-null   string        
 1   release_year   3544 non-null   Int64         
 2   reviewer_name  3560 non-null   string        
 3   comment_count  3305 non-null   float64       
 4   like_count     2578 non-null   float64       
 5   rating         3343 non-null   float64       
 6   review_date    3373 non-null   datetime64[ns]
 7   review         3560 non-null   string        
dtypes: Int64(1), datetime64[ns](1), float64(3), string(3)
memory usage: 253.8 KB
